# **Start & Install Flux**

In [ ]:
# @markdown # Start 💕
# @markdown Click to download Model and Libraries

from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

clear_output()

%cd /content
!git clone -b totoro4 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.28.post2 gradio==3.50.2 python-multipart==0.0.12


clear_output()

button = widgets.Button(description="Done! <3", button_style="success")
display(button)

Button(button_style='success', description='Done! <3', style=ButtonStyle())

---

In [ ]:
# @markdown # LoRA and Flux Models ✨
# @markdown Enter the name of the Hugging Face repository from which the model is loaded.

model_repo = "camenduru/FLUX.1-dev"  # @param {type:"string"}
# @markdown Enter the name of the LoRA on the Hugging Face from which the model is loaded
lora_model = "flux_realism_lora"     # @param {type:"string"}
# @markdown Mark this check is the model/lora is private.
priv = False # @param {type:"boolean"}
# @markdown Enter the Hugging Face token to access private models. (not work sowwy)
hf_token = ""  # @param {type:"string"}

# @markdown Enable Google Drive integration
gdrive = False  # @param {type:"boolean"}

import os
import requests
from tqdm import tqdm
import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro_extras import nodes_flux
from totoro import model_management
import gradio as gr
from IPython.display import clear_output

checkpoint_dir = "/content/TotoroUI/models/checkpoints"
lora_dir = "/content/TotoroUI/models/loras"
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(lora_dir, exist_ok=True)

if gdrive:
    from google.colab import drive
    drive.mount('/content/drive')
    output_dir = "/content/drive/My Drive/silly-image-flux"
else:
    output_dir = "/content/silly-image-flux"

os.makedirs(output_dir, exist_ok=True)

def download_with_token(url_modelo: str, token_hf: str = None, ruta_destino: str = None, is_private: bool = False) -> str:

    # Configure headers according to whether the model is private or public
    headers = {}
    if is_private:
        if token_hf is None:
            raise ValueError("Se requiere token_hf para descargar modelos privados")
        headers["Authorization"] = f"Bearer {token_hf}"

    try:
        response = requests.get(url_modelo, headers=headers, stream=True)
        response.raise_for_status()

        # Get the file name from the URL
        nombre_archivo = os.path.basename(url_modelo)

        # If ruta_destino is a directory, combine with file name
        if ruta_destino is None:
            ruta_completa = nombre_archivo
        else:
            # If it's a directory, add the file name
            if os.path.isdir(ruta_destino):
                ruta_completa = os.path.join(ruta_destino, nombre_archivo)
            else:
                ruta_completa = ruta_destino

        tamaño_total = int(response.headers.get('content-length', 0))

        with open(ruta_completa, 'wb') as f:
            with tqdm(total=tamaño_total, unit='iB', unit_scale=True, desc=f'Downloading {nombre_archivo}') as pbar:
                for datos in response.iter_content(chunk_size=8192):
                    tamaño = f.write(datos)
                    pbar.update(tamaño)

        print(f"\nModel downloaded at: {ruta_completa}")
        return ruta_completa

    except requests.exceptions.RequestException as e:
        raise Exception(f"Error downloading model: {str(e)}")

checkpoint_url = f"https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8-all-in-one.safetensors"
lora_url = f"https://huggingface.co/{model_repo}/resolve/main/{lora_model}.safetensors"

download_with_token(checkpoint_url, hf_token, checkpoint_dir, priv)
download_with_token(lora_url, hf_token, lora_dir, priv)

CheckpointLoaderSimple = NODE_CLASS_MAPPINGS["CheckpointLoaderSimple"]()
LoraLoader = NODE_CLASS_MAPPINGS["LoraLoader"]()
FluxGuidance = nodes_flux.NODE_CLASS_MAPPINGS["FluxGuidance"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    unet, clip, vae = CheckpointLoaderSimple.load_checkpoint("flux1-dev-fp8-all-in-one.safetensors")
def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

@torch.inference_mode()
def generate(positive_prompt, width, height, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip):
    global unet, clip

    clear_output()

    if seed == 0:
        seed = random.randint(0, 18446744073709551615)

    unet_lora, clip_lora = LoraLoader.load_lora(unet, clip, f"{lora_model}.safetensors", lora_strength_model, lora_strength_clip)

    cond, pooled = clip_lora.encode_from_tokens(clip_lora.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    cond = FluxGuidance.append(cond, guidance)[0]

    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet_lora, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet_lora, scheduler, steps, 1.0)[0]
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    decoded = VAEDecode.decode(vae, sample)[0].detach()

    output_filename = os.path.join(output_dir, f"flux_{seed}.png")
    Image.fromarray(np.array(decoded * 255, dtype=np.uint8)[0]).save(output_filename)

    return output_filename

with gr.Blocks(analytics_enabled=False) as demo:
    with gr.Row():
        with gr.Column():
            positive_prompt = gr.Textbox(lines=3, interactive=True, value="A small kitten sits next to a palm tree on a sunny beach, the kitten is about 3 months old, it has light grey fur, its eyes are very large and blue, it looks very playful and cute, around it are a few flowers and palm leaves, the sun's rays shine through the palm tree, creating a very idyllic atmosphere, there are no people around, only the natural landscape of the beach, the kitten looks very alone, but very content. Below it in the sand is written 'by: mikus', and slightly below that 'code: camenduru', a little further away from the text lies a single piece of seaweed, which looks very unusual.", label="Prompt")
            width = gr.Slider(minimum=256, maximum=2048, value=1024, step=16, label="width")
            height = gr.Slider(minimum=256, maximum=2048, value=1024, step=16, label="height")
            seed = gr.Slider(minimum=0, maximum=18446744073709551615, value=0, step=1, label="seed (0=random)")
            steps = gr.Slider(minimum=4, maximum=50, value=20, step=1, label="steps")
            guidance = gr.Slider(minimum=0, maximum=20, value=3.5, step=0.5, label="guidance")
            lora_strength_model = gr.Slider(minimum=0, maximum=1, value=1.0, step=0.1, label="lora_strength_model")
            lora_strength_clip = gr.Slider(minimum=0, maximum=1, value=1.0, step=0.1, label="lora_strength_clip")
            sampler_name = gr.Dropdown(["euler", "heun", "heunpp2", "dpm_2", "lms", "ddim"], label="sampler_name", value="euler")
            scheduler = gr.Dropdown(["normal", "sgm_uniform", "simple", "ddim_uniform"], label="scheduler", value="simple")
            generate_button = gr.Button("Generate")
        with gr.Column():
            output_image = gr.Image(label="Generated image", interactive=False)

    generate_button.click(fn=generate, inputs=[positive_prompt, width, height, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip], outputs=output_image)

demo.queue().launch(inline=False, share=True, debug=True)

In [ ]:
# @markdown # Download image 🖼️
# @markdown Use this button if you have not activated the "gdrive" button.


import shutil
import os
from google.colab import files
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from IPython import get_ipython

# Enter the folder path (can be changed via parameter)
output_dir = "/content/silly-image-flux"  # @param {type:"string"}

clear_output()

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Create the zip file
base = os.path.basename(output_dir)
archive_path = os.path.join(os.path.dirname(output_dir), f"{base}.zip")

try:
    shutil.make_archive(base, 'zip', output_dir)
    print(f"Zip file created: {archive_path}")
except Exception as e:
    print(f"Error creating zip file: {e}")
    get_ipython().system(f"rm -f {archive_path}")
    raise

# Check if the zip file exists before downloading
if os.path.isfile(archive_path):
    print(f"Downloading {os.path.basename(archive_path)}")
    files.download(archive_path)
else:
    print(f"Error: Zip file {archive_path} not found.")


button = widgets.Button(description="Done! <3", button_style="success")
display(button)